# Day 12 
## Part 1
Each question mark has two possible states, so the number of combinations is two to the power of the number of question marks in a row. What's the maximum?

In [9]:
def parse_data(s):
    lines = s.strip().splitlines()
    return [
        (springs, [int(n) for n in ns.split(",")])
        for (springs, ns) in [
            line.split() 
            for line in s.strip().splitlines()
        ]
    ]

data = parse_data(open("input").read())
max(2 ** springs.count("?") for springs, _ in data)

262144

That's tractable individually. How many in total?

In [10]:
sum(2 ** springs.count("?") for springs, _ in data)

5488304

Not _too_ bad. I think I'm just going to check each combination for now.

In [13]:
import itertools

def damaged_springs(springs):
    return [
        len(list(g))
        for k, g in itertools.groupby(springs)
        if k == "#"
    ]

damaged_springs("#....######..#####.")

[1, 6, 5]

Hellish combination of regular expressions and itertools to get each possible string with the unknowns replaced.

In [26]:
import re

def spring_combos(springs):
    n_unknowns = springs.count("?")
    for p in itertools.product(".#", repeat=n_unknowns):
        known_unknown = iter(p)
        yield re.sub("\?", lambda _: next(known_unknown), springs)
    
list(spring_combos("?.?"))

['...', '..#', '#..', '#.#']

In [29]:
def valid_arrangements(springs, damaged_counts):
    return sum(
        1
        for s in spring_combos(springs)
        if damaged_springs(s) == damaged_counts
    )

test_data = parse_data("""???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1""")

[valid_arrangements(s, d) for s, d in test_data]

[1, 4, 1, 1, 4, 10]

In [31]:
def part_1(data):
    return sum(valid_arrangements(s, d) for s, d in data)

assert part_1(test_data) == 21

In [32]:
%%time

part_1(data)

CPU times: user 22.9 s, sys: 36.2 ms, total: 22.9 s
Wall time: 22.9 s


7307

## Part 2
The previous approach was slow anyway and is now intractable. I need to have a think.

Possibly a pruned search will work? I think even that will be too big.

In [38]:
def unfold(springs, damaged_counts):
    return (
        "?".join([springs] * 5),
        list(itertools.chain(damaged_counts * 5))
    )

unfold(*test_data[0])

('???.###????.###????.###????.###????.###',
 [1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3])

Thinking about it it might not be too bad. Create a recursive function that for each initial part of the string that matches the first damaged size adds the total arrangements for the rest.

In [54]:
def valid_arrangements(springs, damaged_counts):
    if len(damaged_counts) == 0:
        return 0 if "#" in springs else 1
    total = 0
    min_string_left = (
        sum(damaged_counts[1:]) # count of damaged
        + len(damaged_counts[2:]) # undamaged gaps
    )
    for i in range(len(springs) - min_string_left - damaged_counts[0] + 1):
        if (
            "." not in springs[i : i + damaged_counts[0]]
            and (
                len(springs[i + damaged_counts[0]:]) == 0
                or springs[i + damaged_counts[0]] in ".?"
            )
        ):
            total += valid_arrangements(
                springs[i + damaged_counts[0] + 1:],
                damaged_counts[1:]
            )
    return total

[valid_arrangements(*x) for x in test_data]

[1, 4, 1, 1, 4, 20]

Nnngh. Wrong for the last one.